Nota generada a partir de [liga](https://www.dropbox.com/s/z465znq3wwao9ad/2.1.Un_poco_de_historia_y_generalidades.pdf?dl=0)

**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_local -p 8888:8888 -d palmoreck/jupyterlab:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_local
```


Documentación de la imagen de docker `palmoreck/jupyterlab:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab).

---

# Temas a considerar en un programa de máquina de alto rendimiento

Para tener un alto performance en un programa de máquina, deben considerarse las siguientes preguntas:

* ¿Qué tanto aprovecha mi programa aspectos como **data reuse** y **data locality**? 

La respuesta nos lleva a pensar en el número de instrucciones por ciclo y el número de ciclos que realiza el procesador. Entiéndase un ciclo por los pasos de leer una instrucción, determinar acciones a realizar por tal instrucción y ejecutar las acciones (ver [liga](https://en.wikipedia.org/wiki/Instruction_cycle)).

* ¿Cómo es mi **data layout** en el almacenamiento? (forma en la que están almacenados o dispuestos los datos)

Dependiendo de la respuesta podemos elegir una arquitectura de computadoras u otra y así también un algoritmo u otro.

* ¿Cuánto **data movement** o **data motion** realiza mi programa? (flujo de datos entre los distintos niveles de jerarquía de almacenamiento o entre las máquinas en un clúster de máquinas)

La respuesta implica analizar el tráfico de datos entre las **jerarquías de almacenamiento** (o máquinas si estamos en un clúster de máquinas) y potenciales **bottlenecks**.

# Herramientas que tenemos a nuestra disposición para un programa de máquina de alto rendimiento

* Vectorización que pueden realizar los procesadores.

* Perfilamiento de código para lograr la eficiencia deseada. Ver [1.6.Perfilamiento.ipynb](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.6.Perfilamiento.ipynb). 

* Programación en lenguajes compilados en lugar de intérpretes (o combinando intérpretes con lenguajes compilados, ver [1.7.Compilar_a_C_Cython](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.7.Compilar_a_C_Cython.ipynb) ).

* Conocimiento de los propósitos con los que fueron diseñados los procesadores para explotar su capacidad. Aquí decidimos si usamos **código secuencial** o **código en paralelo**.

...además necesitamos conocer las diferentes **arquitecturas** que pueden utilizarse para cómputo en paralelo.

* ¿Alguien ya resolvió mi bottleneck?

* Experiencia en el lenguaje de programación seleccionado. 


# Un poco de historia y generalidades del sistema en una computadora

Las componentes fundamentales de un sistema en una computadora pueden simplificarse en:

* Unidades computacionales. En éstas unidades nos interesa la pregunta ¿cuántos cálculos pueden realizar por segundo?

* Unidades de memoria. En éstas unidades nos interesa la pregunta ¿cuántos datos pueden alojar y qué tan rápido puede leerse desde y escribirse hacia las distintas jerarquías?

* Conexiones entre las unidades anteriores. Nos interesa ¿qué tan rápido pueden moverse datos de un lugar a otro?

Con esta simplificación tenemos como ejemplo a la **CPU** como unidad de cómputo conectada tanto a la **RAM** y a un disco duro como dos unidades de memoria y un *bus* que provee las conexiones entre estas partes. Otro ejemplo es considerar que la CPU tiene diferentes unidades de memoria en ella: los cachés tipo L1, L2, L3 y L4* conectadas a la CPU a través de otro *bus*. También la GPU es ejemplo de una unidad de cómputo conectada a una unidades de memoria como RAM y cachés.

*ver [liga](https://es.wikipedia.org/wiki/Caché_(informática)) para información sobre cachés tipo L.

Un dibujo simplificado y basado en una arquitectura de computadoras con nombre [Von Neumann](https://en.wikipedia.org/wiki/Von_Neumann_architecture) que nos ayuda a visualizar lo anterior en la **CPU** es el siguiente:

<img src="https://dl.dropboxusercontent.com/s/txsj5mzxyajbypa/von_Neumann.png?dl=0" heigth="500" width="500">

* La **memoria principal** es una colección de ubicaciones que almacenan datos e instrucciones. Cada ubicación consiste de una dirección (*address*) que se utiliza para accesar a la ubicación y a sus contenidos.

* La **CPU** está dividida en la unidad de control y la unidad aritmética y lógica. Aquí encontramos registers que son áreas o ubicaciones de almacenamiento (de datos, direcciones de memoria e información del estado de ejecución de un programa) de rápido acceso.

* La **interconexión** o *bus* ayuda a la transferencia de datos e instrucciones entre la CPU y la memoria.

**Obs:** 

* En el dibujo no se está presentando los dispositivos de input y output pero sí aparecen en la arquitectura de Von Neumann.

* También no se presentan en el dibujo unidades de almacenamiento como los discos duros pero también aparecen en la arquitectura de Von Neumann. Los discos duros se consideran dentro de las unidades de memoria y la CPU se conecta a ellos mediante un *bus*.

* Si los datos se transfieren de la memoria a la CPU se dice que los datos o instrucciones son leídas y si van de la CPU a la memoria decimos que son escritos a memoria.

* La separación entre la memoria y la CPU genera lo que se conoce como **Von Neumann bottleneck** y tiene que ver con la lectura/escritura y almacenamiento de datos e instrucciones. La interconexión determina la tasa a la cual se accede a éstos.

## Unidades computacionales

Sea una CPU o una GPU, las unidades computacionales toman como input una conjunto de bits (que representan números por ejemplo) y producen otro conjunto de bits (por ejemplo la suma de los números). El performance de éstas unidades se mide en instrucciones por ciclo* y en ciclos por segundo*. Para referencias de *instrucciones por ciclo (IPC) ver [liga](https://en.wikipedia.org/wiki/Instructions_per_cycle) y número de ciclos por segundo (llamado clock rate o clock speed) ver [liga](https://en.wikipedia.org/wiki/Clock_rate).

Entre los rediseños que se han hecho del modelo clásico de Von Neumman para resolver los bottlenecks, mejorar la velocidad y e performance se encuentran:

### 1) Múltiples CPU's o cores

Mientras que incrementar el clock speed en una unidad computacional hace más rápido a un programa, también es importante la medida de IPC. La IPC se puede incrementar vía la vectorización y es típico en procesadores que soportan las instrucciones llamadas **Single Instruction Multiple Data** (SIMD). La vectorización consiste en que dados múltiples datos, el procesador puede trabajar sobre ellos en un instante o tiempo (ejecución en **paralelo**):

<img src="https://dl.dropboxusercontent.com/s/mpfk9xmtq9fm7vm/SIMD.png?dl=0" heigth="500" width="500">

**Comentario:** este tipo de procesadores vinieron a reemplazar al modelo de Von Neumann clásico **Single Instruction Single Data** (SISD) en el que un conjunto de datos se procesaban en un tiempo determinado y no de forma simultánea o en **paralelo**:

<img src="https://dl.dropboxusercontent.com/s/bdx27axhnl3ug5n/SISD.png?dl=0" heigth="300" width="300">

por lo que se decidió transitar de un diseño de hardware secuencial hacia un hardware paralelo: la industria* entre los años $2003-2005$ en lugar de fabricar procesadores monolíticos (clásico Von Neumann) que fueran más rápidos y complejos, decidió fabricar múltiples, simples procesadores, *cores*, en un sólo chip para incrementar el poder de procesamiento, disminuir el Von Neumann bottleneck y aumentar el clock speed. El término *core* hoy en día lo usamos como sinónimo de procesador.

*Esto fue motivado pues desde el año $2002$ el incremento del performance de los procesadores con un sólo CPU fue de un $20\%$ por año vs un $50\%$ por año entre $1986$ y $2002$. Lo anterior se debió a los problemas de la construcción de procesadores monolíticos o de un core relacionados con la disipación del calor por un mayor consumo de energía al hacer más pequeños los transistores.

Un dibujo de una CPU y una GPU que nos ayudan a visualizar máquinas multicore con capacidad de cómputo en paralelo son los siguientes:

<img src="https://dl.dropboxusercontent.com/s/k11qub01w4nvksi/CPU_multicore.png?dl=0" heigth="500" width="500">


**GPU**

<img src="https://dl.dropboxusercontent.com/s/lw9kia12qhwp95r/GPU.png?dl=0" heigth="500" width="500">

**Comentarios:** 

* Un ejemplo de procesadores SIMD son los procesadores vectoriales o en arreglo, ver [liga](https://en.wikipedia.org/wiki/Vector_processor).

* En la práctica se ha visto que simplemente añadir más CPU's o cores al sistema no siempre aumenta la velocidad de ejecución en un programa. Existe una ley que explica lo anterior, la ley de Amdahl, la cual indica que si un programa está diseñado para ejecutarse en múltiples cores y tiene algunas secciones de su código que pueden sólo ejecutarse en un core, entonces éste será el bottleneck del programa. Por ejemplo, si tuviéramos que realizar una encuesta que tarda $1$ min a $100$ personas y tenemos una sola persona, entonces nos tardaríamos $100$ minutos (proceso serial). Si tenemos a $100$ personas entonces nos tardaríamos $1$ minuto en completar todas las encuestas (proceso en paralelo). Pero si tenemos más de $100$ personas, entonces no nos tardaremos menos de $1$ minuto pues las personas "extras" no podrán participar en realizar la encuesta. En este punto la única forma de reducir el tiempo es reducir el tiempo que le toma a una persona encuestar a otra (esta es la parte secuencial del programa). 

* Los sistemas SISD, SIMD y MIMD, presentado a continuación:


<img src="https://dl.dropboxusercontent.com/s/ddze2xuzwn9bh6h/MIMD.png?dl=0" heigth="500" width="500">

son parte de la taxonomía de Flynn (ver [liga](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)) que clasifica a los sistemas dependiendo del número de stream de datos e instrucciones que puede procesar. Ejemplos de sistemas MIMD son máquinas multicore y clústers de máquinas.

### 2) Threading o Hyperthreading

Otra funcionalidad que se les añadió a los procesadores para incrementar la velocidad de ejecución de un proceso* y resolver el bottleneck de Von Neumann fue la capacidad de crear hilos, *threads*, de ejecución en un programa contenidos en un proceso. El llamado *threading* o *hyperthreading* en una CPU o en un core. Básicamente el *threading* permite la ejecución de más de una instrucción en un mismo core "virtualizando" un procesador adicional (el sistema operativo "cree" que en lugar de haber un core hay dos).

*Un proceso es una instancia de un programa que se ejecuta en el procesador y está compuesto por elementos como por ejemplo los bloques de memoria que puede utilizar (los llamados *stack* y *heap*) e información de su estado, entre otros elementos.

Un thread, al igual que un proceso, es una instancia de un programa, se ejecuta en el procesador pero está contenido en el proceso del que salió. Al estar contenido comparte elementos del proceso pero tiene distinto stack de memoria.

La creación de threads a partir de un proceso se le llama *fork* y su unión al proceso se le llama *join*:

<img src="https://dl.dropboxusercontent.com/s/0vnjfdk7fo62m8h/threading.png?dl=0" heigth="400" width="400">

## Unidades de memoria

Su objetivo es el almacenamiento de bits de información. Como ejemplos tenemos la memoria RAM, discos duros o el caché. La gran diferencia entre cada tipo de unidad de memoria es la velocidad a la que pueden leer/escribir datos. Ésta velocidad depende enormemente de la forma en que se leen los datos. Por ejemplo, la mayoría de las unidades de memoria tienen un mejor performance al leer un gran pedazo de información que al leer muchos pedacitos*.


\*Desde el punto de vista de los lenguajes de programación como Python o R, un resultado del manejo automático de memoria en estos lenguajes, es la fragmentación de datos o **data fragmentation** que surge al no tener bloques contiguos de memoria. Esto causa que en lugar de mover todo un bloque contiguo de datos en una sola transferencia a través del *bus* se requieran mover pedazos de memoria de forma individual lo cual causa un mayor tiempo de lectura.

Las unidades de memoria tienen latencia* que típicamente cambia dependiendo de una jerarquía de almacenamiento mostrada en el  dibujo siguiente:

*Entiéndase por latencia el tiempo que le toma a la unidad o dispositivo para encontrar los datos que serán usados por el proceso.

### Jerarquías de almacenamiento

<img src="https://dl.dropboxusercontent.com/s/ahxsnpgp4rjdvw3/jerarquias_de_almacenamiento.png?dl=0" heigth="500" width="500">


la capacidad de almacenamiento disminuye conforme nos movemos hacia arriba en el dibujo: mientras que en disco podemos almacenar terabytes de información, en los registers sólo podemos almacenar bytes o kilobytes. Por el contrario, la velocidad de lectura/escritura disminuye conforme nos movemos hacia abajo: la lectura y escritura en disco es órdenes de veces más tardado que en los registers.

### Caché

Entre las técnicas que tenemos a nuestro alcance para que un algoritmo pueda aprovechar el **data layout** de la información se encuentra el *caching*: el eficiente uso del caché. El caché es una memoria que está físicamente localizada más cercana a los registers del procesador para almacenar datos e instrucciones por lo que pueden ser accesados en menor tiempo que en otras unidades de memoria (como el RAM).

Aunque no tenemos en nuestros lenguajes de programación instrucciones del tipo "carga los datos en el caché" podemos usar los principios de **localidad** y **temporalidad** para mejorar la eficiencia de nuestros algoritmos. Los principios de localidad y temporalidad consisten en que el sistema de memoria tiende a usar los datos e instrucciones que físicamente son cercanos (localidad) y los datos e instrucciones que recientemente fueron usados (temporalidad).


**Preguntas de comprehensión**

1) ¿Qué factores han influido en que desde el 2002-2003 a la fecha, el performance de los procesadores se esté incrementando en un 20% por año vs el 50% de incremento por año que se tenía entre 1986 y 2002?

2) Menciona los componentes y realiza un esquema de una arquitectura von Neumann y descríbelas.

3) Menciona algunas de las tareas de un sistema operativo.

4) ¿Qué es un proceso y de qué consta?

5) ¿Qué es un thread?

6) ¿Qué es el threading? ¿qué ventajas nos da para la programación en un sistema de memoria compartida?

7) ¿Qué es el caché?

8) Nosotros como programadores o programadoras, ¿cómo podemos obtener ventajas del cache?

9) ¿Qué es un cache hit? ¿un cache miss?

10) De acuerdo a la taxonomía de Flynn, ¿qué tipos de arquitecturas existen? Menciona sus características, ventajas /desventajas y ejemplos.

11) Menciona algunas características y ejemplos de:

    a. sistemas de memoria distribuida.
    b. sistemas de memoria compartida.

12) ¿Qué significan los términos concurrencia, paralelo, distribuido?

13) ¿Cuáles son los enfoques que se utilizan para escribir programas en paralelo?

14) Define a cuál enfoque corresponde (de acuerdo a la pregunta 13) cada uno de los siguientes incisos:

    a) Supón que tienes 2 cores y un arreglo de tamaño 100
    
        if(rango_core módulo 2 == 0 )
            operar en los elementos 50 a 99
        else
            operar en los elementos 0 a 49
            
    Donde módulo es una operación que nos devuelve el residuo al dividir un número entre otro.
    
    b) Tenemos tres trabajadores: Aurora, Pedro, Daniel
        
        if(mi_nombre es Pedro)
            lavo el baño
        else
            voy de compras
            
15) En el cómputo en paralelo debemos realizar coordinación entre procesos o threads y considerar el load balancing. Menciona tipos de coordinación que existen y ¿a qué se refiere el load balancing?

16) ¿Cuáles son los pasos a seguir, que de acuerdo a Ian Foster, un@ puede seguir para el diseño de programas en paralelo?


**Referencias:**

1. P. Pacheco, An Introduction to Parallel Programming, Morgan Kaufmann, 2011.

2. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.
